# Production and Deployment

*Authors: Ra Inta*

*Copyright 2019, BH Analytics, LLC*

## Overview

The purpose of this section is to provide an introduction to applying your Python code in a production environment, and deploying and maintaining this code. There are currently many options to achieve these goals; they tend to be very specific towards your particular application. We cover here only a brief overview on this vast topic.

In particular, we will cover:

 *  Packaging scripts
 *  Automation
 *  Containerization and Docker
 
This will provide illumination into the process of how Python imports modules, the differences between a _module_, _package_ and a _library_ and how to automate your Python code. Finally, we will cover the concept of containerization, using Docker, for rapid deployment of applications. 
 
One important note: **we do not cover the security aspects of your deployments**. These are also very dependent on your application and development/deployment environment. Please consult your local security team for best practices regarding this.

## Packaging scripts

So far we have been leveraging libraries from the vast eco-system Python has to offer. After a possible installation process, we make sure we `import` these libraries before using them, and then everything seems to (mostly) 'just work'. 

But _how_ exactly does this work?

### Modules, packages and libraries


Let's take a step back to consider this process. 

Take the widely-used `os` library. How does Python know how/where to retrieve this library?

If we import it, we can query its `__file__` attribute: 

In [1]:
import os

os.__file__

'/home/ra/anaconda3/lib/python3.7/os.py'

This is the location of the Python script containing the code for the `os` module. Note the filename has the _exact_ name as the module we imported.

If we examine a random section of `os.py`, _e.g._ the `makedirs` function:

In [2]:
%less /home/ra/anaconda3/lib/python3.7/os.py

In [ ]:
def makedirs(name, mode=0o777, exist_ok=False):
    """makedirs(name [, mode=0o777][, exist_ok=False])
    Super-mkdir; create a leaf directory and all intermediate ones.  Works like
    mkdir, except that any intermediate path segment (not just the rightmost)
    will be created if it does not exist. If the target directory already
    exists, raise an OSError if exist_ok is False. Otherwise no exception is
    raised.  This is recursive.
    """
    head, tail = path.split(name)
    if not tail:
        head, tail = path.split(head)
    if head and tail and not path.exists(head):
        try:
            makedirs(head, mode, exist_ok)
        except FileExistsError:
            # Defeats race condition when another thread created the path
            pass
        cdir = curdir
        if isinstance(tail, bytes):
            cdir = bytes(curdir, 'ASCII')
        if tail == cdir:           # xxx/newdir/. exists if xxx/newdir exists
            return
    try:
        mkdir(name, mode)
    except OSError:
        # Cannot rely on checking for EEXIST, since the operating system
        # could give priority to other errors like EACCES or EROFS
        if not exist_ok or not path.isdir(name):
            raise 

We can see it looks like any other Python code.

(**Note:** You may be aware that most of Python itself is written in Python and the C language. The latter is for improved performance, but it makes it more difficult to read the original code. Much of the scientific stack, including NumPy and pandas, is written in C)

How did Python know where to look for this module? 

First it looks in your current working directory (`os.getcwd()`). 

If the module you are trying to import is not found there, then it refers to the environment variable `PYTHONPATH`. This is dependent on your operating system (and is a likely source of nightmares and PTSD for anyone who has had to maintain several versions of Python on the same system!) `PYTHONPATH` holds a list of directories in which to look for the file associated with the module in question. 

Finally, if it is still not found, Python will finally look in the default location of the installation binary/executable.



The `from` statement looks for a function or class definition in the module, and imports only this. 

Taking the `os` example, we already know what the `makedirs` function looks like: 

In [5]:
from os import makedirs 

help(makedirs)

Help on function makedirs in module os:

makedirs(name, mode=511, exist_ok=False)
    makedirs(name [, mode=0o777][, exist_ok=False])
    
    Super-mkdir; create a leaf directory and all intermediate ones.  Works like
    mkdir, except that any intermediate path segment (not just the rightmost)
    will be created if it does not exist. If the target directory already
    exists, raise an OSError if exist_ok is False. Otherwise no exception is
    raised.  This is recursive.



This single-file-per-module approach is fine for relatively simple projects. However a single file may become unweildy for large, complex applications.  It can make more sense to divide the project according to specific purposes. But what if we wish to retain the convenience of a single `import` statement? This is the concept of a _package_.

A package is a directory of Python modules. If we query, say, the pandas package:

In [ ]:
import pandas as pd

In [4]:
pd.__version__

'0.24.2'

In [5]:
pd.__file__

'/home/ra/anaconda3/lib/python3.7/site-packages/pandas/__init__.py'

The purpose of the `__init__.py` (if spoken aloud, this may be abbreviated to "dunder init"; "dunder" meaning here "double underscore") file is to co-ordinate the project, telling the import function where to access each module. It often contains shortened aliases to abstract away some of the complexities of the project organization.

Continuing with the pandas example, let's say we want to read the code for `pd.read_csv()`. 

Take the following excerpt from the `__init__.py` file:

In [9]:
from pandas.io.api import *  # Line 42 from this version of pandas (see above). 

# Note the star import---this is one of the few justified uses of this syntax. Do not do this for most end-user applications!  

This is a reference to an import from the pandas I/O module, `pandas/io/api.py`. 

From there, we're told:

In [ ]:
from pandas.io.parsers import read_csv, read_fwf, read_table  # Line 15

And in `pandas/io/parsers.py`:

In [19]:
 read_csv = _make_parser_function('read_csv', default_sep=',')   # Line 709

 #Where:

 def _make_parser_function(name, default_sep=','):    # Line 528
     pass
     # prepare read_table deprecation
     if name == "read_table":
         sep = False
     else:
         sep = default_sep
     def parser_f(filepath_or_buffer,
                  sep=sep,
                  delimiter=None,
                  # Column and Index Locations and Names
                  header='infer',
                  names=None,
                  index_col=None,
                  usecols=None,
                  squeeze=False,
                  prefix=None,
                  mangle_dupe_cols=True):
        pass 
                  #etc.

Hence the pandas package is organized thus:

    pandas/__init__.py

        core/__init__.py

        io/__init__.py
          api.py
          parsers.py
    etc.

So, in summary: to find `pd.read_csv()`, the interpreter first went to `pandas/__init__.py`, which told it to look in `pandas/io/api.py`, which then said the function it was looking for was really in `pandas/io/parsers.py`. Except, `read_csv()` turns out to be a particular application of the `_make_parser()` function. Phew! 

Although it took a while to trace this back, this way of organizing projects is particularly useful for two reasons:
 1.  The developers can divide the project into distinct folders/sub-projects (and likely be able to divide labor accordingly); and
 1.  All of this complexity is abstracted away from the end-user.
 
Would you have initially thought there was this much complexity when you called `pd.read_csv()`?

### Package distribution

You can create your own packages by constructing a directory structure similarly to that shown here. Once your internal workflow has been codified in this way, it is likely to be a productivity multiplier. Your colleagues don't have to 'reinvent the wheel' and can potentially contribute and improve your project very naturally.

But how do you share this hard work? 

There are many avenues for distribution of Python packages. Assuming you adhere to a system of version control, such as `git`, `svn` or `mercurial` (this could be a whole other module!), you may simply share the code repository (or 'repo' in the modern slang) via internal servers. A more scalable and distributed system is to host your repos online. This can be private or public. Popular avenues for this are `GitHub` (https://github.com/), `GitLab` (https://gitlab.com) or `BitBucket` (https://bitbucket.org/). 

Of course, you most likely didn't install many of your favorite Python libraries from any of these platforms. Most of _these_ are hosted on the Python Packaging Index, PyPI:

https://pypi.org/

This is a more 'official' set of repos, where there are a few more conditions in order to be hosted, including a properly formatted `setup.py` and a license type for your repository. 

If you installed a library using `pip` or `easy_install`, then you accessed PyPI. If you installed via an Anaconda distribution, they did all this for you! 

### What's a Python library?
You may see references to a Python _library_. There is no formal definition for this; it essentially refers to a package.

## Automation
One of the biggest advantages of using a scripting language such as Python is its capacity to automate tasks. In other words, to periodically execute instructions with little or no human intervention.

In fact, there is a highly popular book called "Automate the Boring Stuff with Python: practical programming for total beginners," by Al Sweigart. It is freely available, or for purchase, at https://automatetheboringstuff.com . 

Automation leads to productivity gains by freeing up valuable developers, and also potentially opens the door for scaling tasks. A single humans' time and cognitive half-life is finite; however, once you can run a simple script, there's often little stopping you and others from running frequently. This is doubly true for 'headless' devices, _i.e._ computing systems that do not normally function with much interactive elements such as a screen. This is the case for the majority of IoT (Internet of Things) devices.

Hence one of the largest productivity gains you are likely to see in your work environment is acheiving automation of your work tasks!

There are a number of different means to automate Python scripts in particular. Each requires a _scheduler_. 

Perhaps the simplest approach is to use the scheduler built into the operating system you have your code installed on. At least 2/3 of modern web servers run Linux, [as does every single supercomputer on the TOP500 list](https://top500.org/). We will look at how to automate a Python script to run every two minutes on a Linux/MacOS system, using the `cron` utility. 

On Windows, there is the Windows Task Scheduler to perform an equivalent task (although you will need to create an associated BAT file first; a DOS batch file used to execute commands within Windows. These are commands you would type into the command prompt). 

We will cover how to apply `cron` jobs in particular in class.

## Containerization and Docker

In the above section, we saw how to automate the execution of a Python script. 

What if we wanted to port this application so that it executed, for example, remotely, rather than on your personal computer, or an IoT device. Isn't this what a server is for?

However, maintaining your own server can be time-consuming and reduces portability of your applications and code. Someone (the sysadmin) is obliged to keep whatever operating system updated, and the power and connectivity needs to be maintained. 

One solution to this is to deploy a virtual machine (VM), enabling the developer to distribute, and potentially scale, their application. Although a more portable solution than supporting your own server farm, you still have to maintain an operating system and all the associated overhead, even though the physical is virtualized.

This is where the concept of _containerization_ comes in. A _container_ is a standardized unit supplied with just the right resources the software needs to run. This standardized interface encourages portable and scalable code, allowing the developer to rapidly deploy an application.

Conceptually, this is an exension to the virtualization process leading to a VM, but applying to the operating system itself. This can lead to performance improvements because of the decreased overheads.

In particular, Docker is effectively an operating system for these containers. Like Python it is widely used, and has been around since 2013, so there is a rich eco-system surrounding the framework.

Docker is designed to be lightweight and secure (although there are a number of important security considerations).


---

Why use Docker

Using Docker lets you ship code faster, standardize application operations, seamlessly move code, and save money by improving resource utilization. With Docker, you get a single object that can reliably run anywhere. Docker's simple and straightforward syntax gives you full control. Wide adoption means there's a robust ecosystem of tools and off-the-shelf applications that are ready to use with Docker.
100x100_benefit_deployment1
Ship More Software Faster

Docker users on average ship software 7x more frequently than non-Docker users. Docker enables you to ship isolated services as often as needed.
100x100_benefit_tools
Standardize Operations

Small containerized applications make it easy to deploy, identify issues, and roll back for remediation.
100x100_benefit_migration
Seamlessly Move

Docker-based applications can be seamlessly moved from local development machines to production deployments on AWS.
100x100_benefit_lowcost-affordable
Save Money

Docker containers make it easier to run more code on each server, improving your utilization and saving you money.



When to use Docker

You can use Docker containers as a core building block creating modern applications and platforms. Docker makes it easy to build and run distributed microservices architecures, deploy your code with standardized continuous integration and delivery pipelines, build highly-scalable data processing systems, and create fully-managed platforms for your developers.
100x100_benefit_ccontainers
Microservices

Build and scale distributed application architectures by taking advantage of standardized code deployments using Docker containers.
100x100_benefit_delivery
Continuous Integration & Delivery

Accelerate application delivery by standardizing environments and removing conflicts between language stacks and versions.
AWS_Benefit Icon_AutomatedOperations
Data Processing

Provide big data processing as a service. Package data and analytics packages into portable containers that can be executed by non-technical users.
100x100_benefit_get-started-2
Containers as a Service

Build and ship distributed applications with content and infrastructure that is IT-managed and secured.



Q: What is the difference between Docker Swarm, Kubernetes, and Amazon ECS?

When you want to run lots of Docker containers, orchestration tools like Docker Swarm, Kubernetes, and Amazon Elastic Container Service (ECS) make it possible to start, stop, and monitor thousands (or millions) of containers. 

Docker Swarm is container orchestration software made by Docker that you run and manage yourself. Kubernetes is a popular open source, community maintained container orchestration software that you run and manage yourself. Amazon EKS makes it easier to run Kubernetes on AWS by providing managing the Kubernetes control plane for your containers. Amazon ECS is a fully managed AWS service that makes it easy to run containers on AWS with deep integrations to AWS services such as VPC, load balancing, service discovery, and IAM roles.

---

### Installing Docker


 Install necessary certificates to download Docker images securely

In [ ]:
%%bash
sudo apt-get update
sudo apt-get install -y apt-transport-https ca-certificates wget software-properties-common

 Get the cryptographic (GPG) key for associated with Docker

In [ ]:
%%bash
wget https://download.docker.com/linux/debian/gpg
sudo apt-key add gpg

Add the official Docker repo to the packages searched by apt-get

In [ ]:
%%bash
echo "deb [arch=amd64] https://download.docker.com/linux/debian $(lsb_release -cs) stable" | sudo tee -a /etc/apt/sources.list.d/docker.list

sudo apt-get update

sudo apt-cache policy docker-ce

ra  sudo apt-cache policy docker-ce

docker-ce:

Installed: 5:19.03.1~3-0~debian-stretch

Candidate: 5:19.03.1~3-0~debian-stretch

Version table:

5:19.03.1~3-0~debian-stretch 500

500 https://download.docker.com/linux/debian stretch/stable amd64 Packages

100 /var/lib/dpkg/status

5:19.03.0~3-0~debian-stretch 500

500 https://download.docker.com/linux/debian stretch/stable amd64 Packages

5:18.09.8~3-0~debian-stretch 500

500 https://download.docker.com/linux/debian stretch/stable amd64 Packages

5:18.09.7~3-0~debian-stretch 500

500 https://download.docker.com/linux/debian stretch/stable amd64 Packages

5:18.09.6~3-0~debian-stretch 500

500 https://download.docker.com/linux/debian stretch/stable amd64 Packages

5:18.09.5~3-0~debian-stretch 500

500 https://download.docker.com/linux/debian stretch/stable amd64 Packages

5:18.09.4~3-0~debian-stretch 500

500 https://download.docker.com/linux/debian stretch/stable amd64 Packages 

5:18.09.3~3-0~debian-stretch 500

### Install Docker

In [ ]:
%%bash
sudo apt-get -y install docker-ce

 ### Managing Docker services

start Docker service

In [ ]:
%%bash
sudo systemctl start docker

Stop Docker service

In [ ]:
%%bash
sudo systemctl stop docker

Restart Docker service

In [ ]:
%%bash
sudo systemctl restart docker

Check Docker service status

In [ ]:
%%bash
sudo systemctl status docker

ra  sudo systemctl status docker

● docker.service - Docker Application Container Engine

   Loaded: loaded (/lib/systemd/system/docker.service; enabled; vendor preset: enabled)

   Active: active (running) since Wed 2019-07-31 19:53:34 CDT; 40min ago

     Docs: https://docs.docker.com

 Main PID: 833 (dockerd)

    Tasks: 9

   Memory: 124.4M

      CPU: 737ms

   CGroup: /system.slice/docker.service

           └─833 /usr/bin/dockerd -H fd:// --containerd=/run/containerd/containerd.sock



Jul 31 19:53:31 carter dockerd[833]: time="2019-07-31T19:53:31.401088436-05:00" level=warning msg="Your kernel does not support swap memory limit"

Jul 31 19:53:31 carter dockerd[833]: time="2019-07-31T19:53:31.401544895-05:00" level=warning msg="Your kernel does not support cgroup rt period"

Jul 31 19:53:31 carter dockerd[833]: time="2019-07-31T19:53:31.401840993-05:00" level=warning msg="Your kernel does not support cgroup rt runtime"

Jul 31 19:53:31 carter dockerd[833]: time="2019-07-31T19:53:31.402205417-05:00" level=info msg="Loading containers: start."

Jul 31 19:53:33 carter dockerd[833]: time="2019-07-31T19:53:33.871573710-05:00" level=info msg="Default bridge (docker0) is assigned with an IP address 172.17.0.0/16. Daemon

Jul 31 19:53:33 carter dockerd[833]: time="2019-07-31T19:53:33.983973375-05:00" level=info msg="Loading containers: done."                                                  #Jul 31 19:53:34 carter dockerd[833]: time="2019-07-31T19:53:34.465936142-05:00" level=info msg="Docker daemon" commit=74b1e89 graphdriver(s)=overlay2 version=19.03.1

Jul 31 19:53:34 carter dockerd[833]: time="2019-07-31T19:53:34.481634518-05:00" level=info msg="Daemon has completed initialization"

Jul 31 19:53:34 carter systemd[1]: Started Docker Application Container Engine.

Jul 31 19:53:34 carter dockerd[833]: time="2019-07-31T19:53:34.566843704-05:00" level=info msg="API listen on /var/run/docker.sock"

Autostart Docker upon reboot

In [ ]:
%%bash
sudo systemctl enable docker

Run the nice default hello-world image

In [ ]:
%%bash
sudo docker run hello-world

ra  sudo docker run hello-world

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/

### Check your installation

In [ ]:
%%bash 
docker --version

> Docker version 19.03.1, build 74b1e89

Get information about your Docker image:

In [ ]:
%%bash
docker info

 ra  docker info

Client:

 Debug Mode: false



Server:

 Containers: 3

  Running: 0

  Paused: 0

  Stopped: 3

 Images: 1

 Server Version: 19.03.1

 Storage Driver: overlay2

  Backing Filesystem: extfs

  Supports d_type: true

  Native Overlay Diff: true

 Logging Driver: json-file

 Cgroup Driver: cgroupfs

 Plugins:

  Volume: local

  Network: bridge host ipvlan macvlan null overlay

  Log: awslogs fluentd gcplogs gelf journald json-file local logentries splunk syslog

 Swarm: inactive

 Runtimes: runc

 Default Runtime: runc

 Init Binary: docker-init

 containerd version: 894b81a4b802e4eb2a91d1ce216b8817763c29fb

 runc version: 425e105d5a03fabd737a126ad93d62a9eeede87f

 init version: fec3683

 Security Options:

  seccomp

   Profile: default

 Kernel Version: 4.9.0-8-amd64

 Operating System: Debian GNU/Linux 9 (stretch)

 OSType: linux

 Architecture: x86_64

 CPUs: 1

 Total Memory: 7.801GiB                                                                                                                                                     # Name: carter

 ID: S47H:35SR:TDEM:VJCN:ZKQV:RIBH:VIHK:GQ6L:UUEI:CSBD:O7OE:IM3E                                                                                                            # Docker Root Dir: /var/lib/docker

 Debug Mode: false

 Registry: https://index.docker.io/v1/ 

 Labels:

 Experimental: false

 Insecure Registries:

  127.0.0.0/8

 Live Restore Enabled: false



WARNING: No swap limit support

 ra  

List all your current Docker images:

In [ ]:
%%bash
docker image ls

 ra  docker image ls

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE

hello-world         latest              fce289e99eb9        7 months ago        1.84kB

 ra  

List all the containers:

In [ ]:
%%bash
docker container ls --all

 ra  docker container ls --all

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS                      PORTS               NAMES

9eec5f49fd27        hello-world         "/hello"            36 minutes ago      Exited (0) 36 minutes ago                       clever_lichterman

886424febc43        hello-world         "/hello"            25 hours ago        Exited (0) 25 hours ago                         optimistic_leakey

 ra  

In [ ]:
%%bash
sudo docker images

> docker rmi 886424febc43

### Security considerations
You may have noticed all the commands so far have defaulted to running as root (sudo).

<font color="red">**This is an unsafe default and should be changed**</font>.

There have been documented cases in the wild where attackers were able to elevate their privileges on a server by exploiting this security flaw.

Create a new group and add users to it:

In [ ]:
%%bash
sudo groupadd docker

sudo useradd ra

sudo usermod -aG docker ra

So now you can run Docker with restricted ('normal user') privileges.  

### Creating a Dockerfile
There is a means to automatically assign features to your application. This is achieved using a `Dockerfile`. It is literally a textfile with this name with specifications on how to build the image, based on requirements and the application itself.

In [ ]:
%%bash
mkdir -p docker_test
cd docker_test/

Edit and save the Dockerfile. This is literally a text-file conforming to the following format. 

Docker builds images in layers. 

We want to build a Python 3.7 app, so we'll make use of a pre-compiled base image, from the official list of Docker pre-built images:
https://hub.docker.com/search/?q=&type=image&image_filter=official 

In [14]:
! cat code/docker_test/Dockerfile

# Use an official Python runtime as a parent image
FROM python:3.7-stretch

# Set the working directory to /app
WORKDIR /app

# Copy the current directory contents into the container at /app
COPY . /app

# Install any needed packages specified in requirements.txt
RUN pip install --trusted-host pypi.python.org -r requirements.txt

# Make port 80 available to the world outside this container
EXPOSE 80

# Define environment variable
ENV NAME World

# Run app.py when the container launches
CMD ["python", "app.py"]



Edit your  `requirements.txt` in the same directory. 

Here, we'll just make use of NumPy, so it has one single line:

> Numpy


Pure poetry.

Finally, you'll need to write your code in a format that is executable from a single source. The convention is to name the file `app.py`.

I've written a small game where you fight a mythical beast known as a Grue, and you get to attack it by rolling a twenty-sided die.

 Build your app

In [ ]:
%%bash
docker build --tag=roll20 .

ra  docker_test  docker build --tag=roll20 .

Sending build context to Docker daemon  13.82kB

Step 1/7 : FROM python:3.7-stretch

3.7-stretch: Pulling from library/python

a4d8138d0f6b: Pull complete

dbdc36973392: Pull complete

f59d6d019dd5: Pull complete

aaef3e026258: Pull complete

6e454d3b6c28: Pull complete

47c95b44ab24: Pull complete

5570e9404146: Pull complete

281654452bf7: Pull complete

ea8e7ce389f6: Pull complete

Digest: sha256:99a16ef43ba12811a369a2912d8c73b0ebc69aed55b327653185ba1330e3121c

Status: Downloaded newer image for python:3.7-stretch

 ---> 46ccb963c04a

Step 2/7 : WORKDIR /app

 ---> Running in 2d61c8c30d47

Removing intermediate container 2d61c8c30d47

 ---> afe6111ba6db

Step 3/7 : COPY . /app

 ---> 33de3c146ced

Step 4/7 : RUN pip install --trusted-host pypi.python.org -r requirements.txt

 ---> Running in 67f6423f75ed

Collecting Numpy (from -r requirements.txt (line 1))

  Downloading https://files.pythonhosted.org/packages/05/4b/55cfbfd3e5e85016eeef9f21c0ec809d978706a0d60b62cc28aeec8c792f/numpy-1.17.0-cp37-cp37m-manylinux1_x86_64.whl (20.3MB)

Installing collected packages: Numpy

Successfully installed Numpy-1.17.0                                                                                                                                         #Removing intermediate container 67f6423f75ed

 ---> a377775d78d2                                                                                                                                                          #Step 5/7 : EXPOSE 80

 ---> Running in a47e882fff57

Removing intermediate container a47e882fff57

 ---> 7ce058f2a24d

Step 6/7 : ENV NAME World

 ---> Running in 94af2c3d890e

Removing intermediate container 94af2c3d890e

 ---> 7e616d4dae80

Step 7/7 : CMD ["python", "app.py"]

 ---> Running in f79a3cee87bc

Removing intermediate container f79a3cee87bc

 ---> 399918acce75

Successfully built 399918acce75

Successfully tagged roll20:latest

 ra  docker_test  

 Check your app was installed by checking your local Docker image registry:

In [ ]:
%%bash
docker image ls

 ra  docker_test  docker image ls

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE

roll20              latest              399918acce75        2 minutes ago       1.05GB

python              3.7-stretch         46ccb963c04a        4 days ago          941MB

hello-world         latest              fce289e99eb9        7 months ago        1.84kB

 ra  docker_test  

In [ ]:
%%bash
docker run roll20

You can run this with port settings with the -p flag too
 
If you update just the app code, you can re-build the Docker image.
This is not as resource-intensive as the initial build; only necessary changes are
brought in. 

After updating `app.py`:

In [ ]:
%%bash
docker build --tag=roll20 .

 ra  docker_test  docker build --tag=roll20 .

Sending build context to Docker daemon  14.85kB

Step 1/7 : FROM python:3.7-stretch

 ---> 46ccb963c04a

Step 2/7 : WORKDIR /app

 ---> Using cache

 ---> afe6111ba6db

Step 3/7 : COPY . /app

 ---> 045ec16d8ef9

Step 4/7 : RUN pip install --trusted-host pypi.python.org -r requirements.txt

 ---> Running in 43e19094cefc

Collecting Numpy (from -r requirements.txt (line 1))

  Downloading https://files.pythonhosted.org/packages/05/4b/55cfbfd3e5e85016eeef9f21c0ec809d978706a0d60b62cc28aeec8c792f/numpy-1.17.0-cp37-cp37m-manylinux1_x86_64.whl (20.3MB)

Installing collected packages: Numpy

Successfully installed Numpy-1.17.0

Removing intermediate container 43e19094cefc

 ---> 163207aac796

Step 5/7 : EXPOSE 80

 ---> Running in 8aa0e71c200a

Removing intermediate container 8aa0e71c200a

 ---> b050fbb7e37d

Step 6/7 : ENV NAME World

 ---> Running in 106425262fb7

Removing intermediate container 106425262fb7

 ---> 74f217a1d060

Step 7/7 : CMD ["python", "app.py"]

 ---> Running in 016daabdcdbf

Removing intermediate container 016daabdcdbf

 ---> ccdf3af1d563

Successfully built ccdf3af1d563

Successfully tagged roll20:latest

 ra  docker_test  

 You can get a lot of details about your image using `inspect`:

In [ ]:
%%bash
docker inspect roll20

 ra  ⋯  notebooks  code  docker_test  docker inspect roll20

[

    {

        "Id": "sha256:ccdf3af1d5639c44a2b2237415c3d0eb8d3181f04b3d7297bc5a7a24ba3ac119",

        "RepoTags": [

            "roll20:latest"

        ],

        "RepoDigests": [],

        "Parent": "sha256:74f217a1d0607f53bc7ff0ded59c5260e3b6cdee6d7201bedfcde40bf9d50595",

        "Comment": "",

        "Created": "2019-08-03T22:21:30.841261217Z",

        "Container": "016daabdcdbf44dd66aac37783f216ba034e25f5bc7592c43585337a88ab94c4",

---

In a distributed application, different pieces of the app are called “services”. For example, if you imagine a video sharing site, it probably includes a service for storing application data in a database, a service for video transcoding in the background after a user uploads something, a service for the front-end, and so on.

Services are really just “containers in production.” A service only runs one image, but it codifies the way that image runs—what ports it should use, how many replicas of the container should run so the service has the capacity it needs, and so on. Scaling a service changes the number of container instances running that piece of software, assigning more computing 
 resources to the service in the process.

Luckily it’s very easy to define, run, and scale services with the Docker platform -- just write a docker-compose.yml file.

---

### Initialize a Docker Swarm

In [ ]:
%%bash
docker swarm init

 ra  docker_test  docker swarm init

Swarm initialized: current node (rf2mpr85sxew8crlg4s5zv71s) is now a manager.



To add a worker to this swarm, run the following command:



    docker swarm join --token SWMTKN-1-33jpx3rgyhnm4z2yeiftw2lvdyclhqjqeaoyz7w2ysg4om5c76-1inpnveoah0zjd01gopt4n20o 10.0.2.15:2377



To add a manager to this swarm, run 'docker swarm join-token manager' and follow the instructions.

ra  docker_test  

 ### Deploy the app
 We have to call it something! Let's call this one `attack_the_grue`:

In [ ]:
%%bash
docker stack deploy -c docker-compose.yml attack_the_grue

 ra  docker_test  docker stack deploy -c docker-compose.yml attack_the_grue

Creating network attack_the_grue_default

Creating service attack_the_grue_roll20

 ra  docker_test  

Check all the services running:

In [ ]:
%%bash
docker service ls

 ra  docker_test  docker service ls

ID                  NAME                     MODE                REPLICAS            IMAGE               PORTS

uzxy32f944ye        attack_the_grue_roll20   replicated          0/5                 ra/roll20:latest                                                                       

ra  docker_test  

Alternatively:

In [ ]:
%%bash
docker container ls -q

Show all tasks on a stack:

In [ ]:
%%bash
docker stack ps attack_the_grue 

### Scale up your app

---

You can scale the app by changing the replicas value in docker-compose.yml, saving the change, and re-running the docker stack deploy command:

---

In [4]:
%%bash
docker stack deploy -c docker-compose.yml attack_the_grue

open docker-compose.yml: no such file or directory


CalledProcessError: Command 'b'docker stack deploy -c docker-compose.yml attack_the_grue\n'' returned non-zero exit status 1.

Tear down the app and the swarm

In [ ]:
%%bash
docker stack rm attack_the_grue

In [ ]:
%%bash
docker swarm leave --force

ra  docker_test  docker swarm leave --force

Node left the swarm.

ra  docker_test  

How easy was that? 

You can scale an application using this container model.

### Swarm clusters and the stack

A _swarm_ is all the combined resources running your Docker instances. It's a great mental image. These instances need to be managed.
Swarm managers take intructions on how to run this ensemble of containers. They act as administrators for the spun-up Docker images.

This is the cluster version of the single-instance you have been running up until now.

A _stack_ is all the services that share dependencies and can be co-ordinated together. This, in principle, can organize an entire application, although multiple stacks may also be required.

If you run your Docker application on the cloud, you may need to open ports from your image to services on cloud machines. This is if your app requires communication to an SSH or web service, or to communicate with a database.

Typical port numbers for each service are:


| Service | Type  | Protocol | Port |
| --- | --- | --- | --- |
| Web         | HTTP  | TCP | 80 |
| Visualizer  | HTTP  | TCP | 8080 |
| Redis       | TCP   | TCP | 6379 |

---

### More resources

 The Docker Cheatsheet website:
 https://dockercheatsheet.painlessdocker.com/

 On GitHub:
https://github.com/wsargent/docker-cheat-sheet

 Docker's website:
https://www.docker.com/get-started        

# Conclusion

Here, we covered what goes into packaging a Python project, including the differences between a Python _module_, a _package_ and a _library_ (what was the difference?)

We also looked at automation of Python scripts.

Finally, we looked at how to build and deploy Python scripts rapidly, using the popular containerization framework Docker.